In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('data/train.csv')
df.head()

,Rescue_ID,Date_TimeCaught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,Tag_1,...,Lost_Tags,T_Number,CCL_cm,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,ReleaseSite,Date_TimeRelease
0,2000_RE_0060,2000-12-22,Researcher_25,CaptureSite_0,Ocean,Net,Fisher_1072,LandingSite_CaptureSiteCategory_2,Species_6,CC00147,...,NaN,NaN,64.70,62.60,NaN,Unknown,algae at rear of shell,Released,ReleaseSite_50,22/12/00
1,2001_RE_0187,2001-10-28,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_520,LandingSite_CaptureSiteCategory_2,Species_6,W442,...,NaN,NaN,35.85,31.35,NaN,Unknown,multiple b's on front flippers& a lot of alga...,Released,ReleaseSite_62,28/10/01
2,2001_RE_0197,2001-11-01,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_1669,LandingSite_CaptureSiteCategory_2,Species_5,KE0376,...,NaN,NaN,51.80,49.20,NaN,Unknown,clean,Released,ReleaseSite_50,01/11/01
3,2002_RE_0031,2002-03-11,Researcher_32,CaptureSite_0,Ocean,Net,Fisher_1798,LandingSite_CaptureSiteCategory_2,Species_6,CC00302,...,NaN,NaN,60.50,59.00,NaN,Unknown,1 b 3 CS+ calcerous algae at rear end of shell...,Released,ReleaseSite_50,11/03/02
4,2002_RE_0118,2002-08-08,Researcher_25,CaptureSite_0,Ocean,Beached,Fisher_1918,LandingSite_CaptureSiteCategory_2,Species_5,NotTagged_0113,...,NaN,NaN,34.70,33.00,NaN,Unknown,very lively+ right eye is hanging out + swolle...,Released,ReleaseSite_62,08/08/02


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rescue_ID              18062 non-null  object 
 1   Date_TimeCaught        18062 non-null  object 
 2   Researcher             18062 non-null  object 
 3   CaptureSite            18062 non-null  object 
 4   ForagingGround         18062 non-null  object 
 5   CaptureMethod          18062 non-null  object 
 6   Fisher                 18062 non-null  object 
 7   LandingSite            18062 non-null  object 
 8   Species                18062 non-null  object 
 9   Tag_1                  17937 non-null  object 
 10  Tag_2                  4911 non-null   object 
 11  Lost_Tags              925 non-null    object 
 12  T_Number               38 non-null     object 
 13  CCL_cm                 18038 non-null  float64
 14  CCW_cm                 18035 non-null  float64
 15  We

In [9]:
df.shape

(18062, 21)

This is EDA by Sherzad.

In [5]:
keep_cols = [
    'Date_TimeCaught',
    'CaptureSite',
    #'ForagingGround',
   # 'Species'
]
df = df[keep_cols].copy()
df.head()

,Date_TimeCaught,CaptureSite
0,2000-12-22,CaptureSite_0
1,2001-10-28,CaptureSite_0
2,2001-11-01,CaptureSite_0
3,2002-03-11,CaptureSite_0
4,2002-08-08,CaptureSite_0


In [6]:
df.Date_TimeCaught = pd.to_datetime(df.Date_TimeCaught)
df['CalendarWeek'] = df['Date_TimeCaught'].dt.isocalendar().week
df['Year'] = df['Date_TimeCaught'].dt.year

In [7]:
df

,Date_TimeCaught,CaptureSite,CalendarWeek,Year
0,2000-12-22,CaptureSite_0,51,2000
1,2001-10-28,CaptureSite_0,43,2001
2,2001-11-01,CaptureSite_0,44,2001
3,2002-03-11,CaptureSite_0,11,2002
4,2002-08-08,CaptureSite_0,32,2002
...,...,...,...,...
18057,2018-12-18,CaptureSite_9,51,2018
18058,2018-12-18,CaptureSite_9,51,2018
18059,2018-12-24,CaptureSite_9,52,2018
18060,2018-12-24,CaptureSite_9,52,2018


In [8]:
df.drop('Date_TimeCaught', axis= 1, inplace=True)

In [9]:
df['Count'] = (
    df.groupby(['CaptureSite', 'Year', 'CalendarWeek'])['Year']
      .transform('size')
)
df = (
    df.groupby(['CaptureSite', 'Year', 'CalendarWeek'])
      .size()
      .reset_index(name='Count')
)

In [10]:
df

,CaptureSite,Year,CalendarWeek,Count
0,CaptureSite_0,2000,51,1
1,CaptureSite_0,2001,43,1
2,CaptureSite_0,2001,44,1
3,CaptureSite_0,2002,11,1
4,CaptureSite_0,2002,32,2
...,...,...,...,...
7952,CaptureSite_9,2018,48,6
7953,CaptureSite_9,2018,49,6
7954,CaptureSite_9,2018,50,12
7955,CaptureSite_9,2018,51,5


In [11]:
"""
Extract numeric site IDs from strings and convert them
into a categorical (non-ordinal) variable. This prevents the model from treating
site numbers as continuous or ordered values.
"""
df['CaptureSite'] = df['CaptureSite'].astype(str).str.extract(r'(\d+)').astype(int)
df['CaptureSite'] = df['CaptureSite'].astype('category')
df

,CaptureSite,Year,CalendarWeek,Count
0,0,2000,51,1
1,0,2001,43,1
2,0,2001,44,1
3,0,2002,11,1
4,0,2002,32,2
...,...,...,...,...
7952,9,2018,48,6
7953,9,2018,49,6
7954,9,2018,50,12
7955,9,2018,51,5


## Baseline Model: 3-year prediction

Example:
Row = (site=5, year=2016, week=12)
Take the average of all rows where:

site = 5

week = 12

year ∈ {2013, 2014, 2015}

In [23]:
def baseline_three_year_avg(df):
    """
    Compute a baseline prediction using the average of the same
    (CaptureSite, CalendarWeek) over the previous 3 years.
    
    df must contain: CaptureSite, Year, CalendarWeek, Count
    """

    df = df.copy()
    df = df.sort_values(['CaptureSite', 'Year', 'CalendarWeek'])

    baseline_preds = []

    for idx, row in df.iterrows():
        site = row['CaptureSite']
        year = row['Year']
        week = row['CalendarWeek']

        # past 3 years
        past = df[
            (df['CaptureSite'] == site) &
            (df['CalendarWeek'] == week) &
            (df['Year'].between(year - 3, year - 1))
        ]['Count']

        if len(past) >= 1:
            pred = past.mean()
        else:
            pred = np.nan  # no historical baseline available

        baseline_preds.append(pred)

    df['BaselinePred'] = baseline_preds
    return df


In [37]:
df = baseline_three_year_avg(df)


In [38]:
df.head(10)

,CaptureSite,Year,CalendarWeek,Count,BaselinePred
0,0,2000,51,1,NaN
1,0,2001,43,1,NaN
2,0,2001,44,1,NaN
3,0,2002,11,1,NaN
4,0,2002,32,2,NaN
5,0,2002,43,3,1.0
6,0,2003,24,1,NaN
7,0,2003,33,1,NaN
8,0,2003,36,1,NaN
9,0,2003,39,4,NaN


In [39]:
df.shape

(7957, 5)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7957 entries, 0 to 6359
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   CaptureSite   7957 non-null   category
 1   Year          7957 non-null   int32   
 2   CalendarWeek  7957 non-null   UInt32  
 3   Count         7957 non-null   int64   
 4   BaselinePred  5641 non-null   float64 
dtypes: UInt32(1), category(1), float64(1), int32(1), int64(1)
memory usage: 265.1 KB


In [41]:
df.isna().sum()

CaptureSite        0
Year               0
CalendarWeek       0
Count              0
BaselinePred    2316
dtype: int64

In [33]:
#count NaNs per capture site
#df_with_baseline[df_with_baseline['BaselinePred'].isna()].groupby('CaptureSite').size().sort_values(ascending=False)


In [34]:
# count NaNs per calendar week
#df_with_baseline[df_with_baseline['BaselinePred'].isna()].groupby('CalendarWeek').size().sort_values(ascending=False)


In [42]:
# filter out the rows where baseline exists
mask = df['BaselinePred'].notna()
df_eval = df[mask].copy()

In [46]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae  = mean_absolute_error(df_eval['Count'], df_eval['BaselinePred'])
rmse = np.sqrt(mean_squared_error(df_eval['Count'], df_eval['BaselinePred']))
r2_score = r2_score(df_eval['Count'], df_eval['BaselinePred'])

print(f"Baseline MAE:  {mae:.3f}")
print(f"Baseline RMSE: {rmse:.3f}")
print(f"Baseline R²: {r2_score:.3f}")


Baseline MAE:  1.545
Baseline RMSE: 2.549
Baseline R²: -0.123


In [48]:
sample = pd.read_csv('data/Sample_sub.csv')
sample.head()

,ID,Capture_Number
0,CaptureSite_0_201901,7
1,CaptureSite_0_201902,1
2,CaptureSite_0_201903,5
3,CaptureSite_0_201904,2
4,CaptureSite_0_201905,3
